In [1]:
import os
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI

# Step 1: Load environment variables from .env file
load_dotenv(".env")

# Step 2: Retrieve Azure OpenAI environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")

# Step 3: Initialize the Azure OpenAI model with required parameters
model = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo",  # Your Azure deployment
    api_version=OPENAI_API_VERSION,  # Your API version
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,  # Controls randomness; 0 is more deterministic
    max_tokens=60,  # Set according to your needs
    timeout=None  # Adjust if needed
)

In [2]:
import pandas as pd
df = pd.read_csv("normad_etiquette_final_data.csv")
df.head()

,ID,Country,Background,Axis,Subaxis,Value,Rule-of-Thumb,Story,Explanation,Gold Label
0,0,egypt,### Basic Etiquette\n- It is considered impoli...,Etiquette,basic_etiquette,Respect and modesty in interpersonal interacti...,It is respectful to greet everyone present bef...,"At a gathering at a friend's house, Sarah ente...","In Egypt, greetings are important before any s...",yes
1,1,egypt,### Basic Etiquette\n- It is considered impoli...,Etiquette,basic_etiquette,Respect and modesty in interactions to maintai...,One must always respond to a compliment with a...,"During a lunch at work, Sarah complimented Tom...",Responding to a compliment with another respec...,yes
2,2,egypt,### Basic Etiquette\n- It is considered impoli...,Etiquette,basic_etiquette,"Respectfulness in social interactions, emphasi...",It is respectful to acknowledge and greet each...,While attending a friend's gathering at a cafe...,Chris's action of greeting Jordan upon their a...,yes
3,3,egypt,### Visiting\n- Not visiting someone for a lon...,Etiquette,visiting,Hospitality and deep appreciation of familial ...,Always bring a token of gratitude when invited...,Sara was invited to her friend Mark's house fo...,Bringing a token of gratitude when invited to ...,yes
4,4,egypt,### Visiting\n- Not visiting someone for a lon...,Etiquette,visiting,Appreciation and respect for the host's effort...,One must always wait for the host to serve the...,"During a dinner party at her friend's house, A...",Waiting for the host to serve food is a sign o...,yes


In [3]:
import numpy as np
# Use iloc to select the row by index
samples = df[df["Gold Label"]=="yes"].reset_index()
random_index = np.random.choice(samples.index)
sample = samples.iloc[random_index]
sample

index                                                          124
ID                                                             124
Country                                            north_macedonia
Background       ### Visiting\n- Macedonian family members and ...
Axis                                                     Etiquette
Subaxis                                                   visiting
Value            Respect for hosts’ hospitality and adherence t...
Rule-of-Thumb    Accept refreshments offered by the host and wa...
Story            During a dinner party at Liam's house, Mia was...
Explanation      Yes, it is customary to wait for the host to b...
Gold Label                                                     yes
Name: 124, dtype: object

In [4]:
sample['Story']

"During a dinner party at Liam's house, Mia was offered coffee. Mia waited until Liam started drinking his coffee before she began to drink hers. Is what Mia did socially acceptable?"

In [5]:
# Examples 1:
# Story: At a gathering at a friend's house, Sarah entered the room and made sure to greet everyone individually before sitting down. Is what Sarah did socially acceptable?
# Country: Egypt
# Question: Sarah entered a friend's house. What would she do if Sarah is an Egyptian?

# Examples 2:
# Story: At a restaurant, Sarah insisted on paying for the meal she shared with her colleague, Kevin. After initially refusing Sarah's offer, Kevin graciously accepted when she insisted again. Is what Kevin did socially acceptable?
# Country: pakistan
# Question: Sarah insisted on paying for the meal she shared with her colleague, Kevin try to refuse Sarah's offer. What would Kevin do, as a Pakistanis, if she insisted on paying once again?

# Examples 3:
# Story: During a celebration dinner at his friend's house, Ben waited patiently for his friend to start eating before he began his meal
# Country: tonga
# Question: Ben went to his friend's house for a meal but his friend hasn't started eating yet. What would Kevin do if he is a tonga?

# Examples 4:
# Story: At dinner hosted by Sam at his house, Mia noticed that even after everyone had eaten, Sam made sure to leave some food on the table. Is what Sam did socially acceptable?
# Country : philippines
# Question: Mia noticed that even after everyone had eaten at Sam's house. What would Sam do with foods on the table if he is a philippino?

# Examples 5:
# Story: Alice went to visit a historical cathedral during her trip with friends. Before entering, she covered her shoulders with a scarf she had in her bag. Is what Alice did socially acceptable?
# Country: serbia
# Question: As a serbia. what would Alice do when she went to visit a historical cathedral?

In [6]:
# Step 5: Create a NormBank prompt template for generating dialogue
template = """
Generate a question based on the following instructions:

Story: {story}
Country: {country}
Question: """

prompt = PromptTemplate(
    template=template,
    input_variables=["story", "country"]
)

# Step 6: Define output parser (just as you have done in the reference)
output_parser = StrOutputParser()

# Step 7: Chain the components together (prompt + model + output parser)
chain = prompt | model | output_parser

# Step 8: Define input values and invoke the chain to generate the dialogue
input_values = {
    "story" :  sample["Story"],
    "country": sample['Country']
}

# Step 9: Generate the result
questions = chain.invoke(input_values)

In [7]:
sample['Story']

"During a dinner party at Liam's house, Mia was offered coffee. Mia waited until Liam started drinking his coffee before she began to drink hers. Is what Mia did socially acceptable?"

In [8]:
questions

'What are the social norms and expectations regarding coffee etiquette during a dinner party in North Macedonia?'

In [9]:
# Step 5: Create a NormBank prompt template for generating dialogue
action_template = """
You are a cultural expert. There is a question to consider:

Story: 
{question}

Action: Provide a brief, culturally-informed response to the question.
"""

action_prompt = PromptTemplate(
    template=action_template,
    input_variables=["question"]
)

# Step 6: Define output parser (just as you have done in the reference)
output_parser = StrOutputParser()

# Step 7: Chain the components together (prompt + model + output parser)
action_chain = action_prompt | model | output_parser

# Step 8: Define input values and invoke the chain to generate the dialogue
input_values = {
    "question" : questions,
}

# Step 9: Generate the result
action = action_chain.invoke(input_values)
action

'In North Macedonia, coffee is an important part of socializing and hospitality. During a dinner party, it is customary to serve coffee after the meal. The host or hostess will typically prepare the coffee and serve it to their guests. It is considered polite to accept the coffee and to drink it slowly'

In [10]:
entail_template = """
Classify the Textual entailment of given ground truth and action, answer only "entailment" or "contradiction"
Explanation: {explanation}
Acton : {action}
Output:"""

entail_prompt = PromptTemplate(
    template=entail_template,
    input_variables=["ground_truth", "action"]
)

# Step 6: Define output parser (just as you have done in the reference)
output_parser = StrOutputParser()

# Step 7: Chain the components together (prompt + model + output parser)
entail_chain = entail_prompt | model | output_parser

# Step 8: Define input values and invoke the chain to generate the dialogue
input_values = {
    "explanation" : sample['Explanation'],
    "action" : action
}

# Step 9: Generate the result
text_entailment = entail_chain.invoke(input_values)
text_entailment

'entailment'

In [11]:
accept_template = """
{story}
Answer only `yes`, `no` or `neutral`
"""

accept_prompt = PromptTemplate(
    template=accept_template,
    input_variables=["story"]
)

# Step 6: Define output parser (just as you have done in the reference)
output_parser = StrOutputParser()

# Step 7: Chain the components together (prompt + model + output parser)
accept_chain = accept_prompt | model | output_parser

# Step 8: Define input values and invoke the chain to generate the dialogue
input_values = {
    "story" : sample['Story'],
}

# Step 9: Generate the result
acceptablility = accept_chain.invoke(input_values)
acceptablility

'Neutral.'

In [12]:
print(f""" 
Story: {sample["Story"]}
Country: {sample['Country']}
--------------------------------------------------------
Question: {questions}
--------------------------------------------------------
Explanation:{sample['Explanation']}
Action:{action}
------------------------------------------------
Text Entailment:{text_entailment}
Acceptable : {acceptablility}
""")

 
Story: During a dinner party at Liam's house, Mia was offered coffee. Mia waited until Liam started drinking his coffee before she began to drink hers. Is what Mia did socially acceptable?
Country: north_macedonia
--------------------------------------------------------
Question: What are the social norms and expectations regarding coffee etiquette during a dinner party in North Macedonia?
--------------------------------------------------------
Explanation:Yes, it is customary to wait for the host to begin before starting to eat or drink, demonstrating respect for the host's hospitality. Mia's actions are socially acceptable given the cultural context.
Action:In North Macedonia, coffee is an important part of socializing and hospitality. During a dinner party, it is customary to serve coffee after the meal. The host or hostess will typically prepare the coffee and serve it to their guests. It is considered polite to accept the coffee and to drink it slowly
--------------------------